# 2.8. 部署模型到生产环境中
## 🚄前言
恭喜你！在经过模型微调和评测后，你的开发工作已经接近完成。本节课程将介绍最后一部分的工作：将你的机器人部署到生产环境中。这里的“部署”指部署模型服务，意味着你将学习如何部署本地模型和线上模型，使模型在提供推理服务的基础上，加速推理速度，提高服务并发量。

本节课程会设计两个实践环节，分别帮助你理解模型部署的意义，以及指导你如何部署微调模型。完成这些实验后，你的机器人就算开发完成了。

最后，你还需要为机器人设计一个合适的载体，例如 Web App 或客户端程序。我们也援引了阿里云百炼两个实践案例，希望能给你一些启发。

相信你也和我一样，迫不及待地让新员工用上这款机器人吧！

## 🪶课程目标
学完本节课程后，你将能够：

➔ 了解模型部署的概念、目的和用途；

➔ 掌握本地模型部署方法；

➔ 了解线上模型的快速部署方法；

➔ 了解发布大模型应用的方法；



## 📖 课程目录

- [1. 为什么要部署模型](#🤔-1-为什么要部署模型？)
    - [1.1. 你可能遇到的情况](#11-你可能遇到的情况)
    - [1.2. 什么是模型部署](#12-什么是模型部署)
    - [1.3. 深入理解模型部署](#13-深入理解模型部署)
        - [量化（Quantization）](#量化quantization)
        - [蒸馏（Distillation）](#蒸馏distillation)
        - [缓存（Caching）](#缓存caching)
        - [负载均衡（Load Balancing）](#负载均衡load-balancing)
        - [数据并行和模型并行](#数据并行和模型并行)
        - [水平扩展和垂直扩展](#水平扩展和垂直扩展)
        - [RESTful API与OpenAI API兼容](#restful-api与openai-api兼容)
- [2. 部署本地模型](#🚀-2-部署本地模型)
    - [2.1. 硬件资源与模型参数量](#21-硬件资源与模型参数量)
    - [2.2. vLLM 是什么](#22-vllm-是什么)
    - [2.3. 在 ModelScope Notebook 上使用 vLLM 部署本地模型服务](#23-在-modelscope-notebook-上使用-vllm-部署本地模型服务)
    - [2.4. 阿里云平台选择（ECS、PAI、FC）](#24-阿里云平台选择ecspaifc)
    - [2.5. 使用vLLM部署模型到阿里云 ECS](#25-使用vllm部署模型到阿里云-ecs)
    - [2.6. 部署到阿里云 PAI - EAS](#26-部署到阿里云-pai---eas)
    - [2.7. 部署到阿里云 FC](#27-部署到阿里云-fc)
- [3. 部署线上模型到百炼](#☁️-3-部署线上模型到百炼)
  - [3.1. 百炼模型部署服务](#31-百炼模型部署服务)
  - [3.2. 部署操作步骤](#32-部署操作步骤)
  - [3.3. 模型部署 API](#33-模型部署-api)
- [4.拓展阅读：让机器人开始“上岗”](#🤖-4拓展阅读让机器人开始上岗)
   

## 🤔 1. 为什么要部署模型？

### 1.1. 你可能遇到的情况

想象一下，当你的新人答疑机器人第一次上线时，一切看起来都很顺利。可是，每天早上的忙碌时段，当几十位新员工同时使用机器人查询各类问题时，情况开始变得复杂。系统开始变慢，部分查询响应时间变长，甚至有些请求被直接丢弃。这种状况不仅影响了新员工的体验，还让管理层质疑机器人的可靠性。

以下是一个小实验，可以帮助你理解高负载下可能遇到的问题：

#### 小实验：从低负载到高负载理解机器人响应的成功率与平均响应时间

本实验旨在帮助你理解在不同峰谷时段（高并发和低并发）下，机器人响应的成功率和平均响应时间会如何变化。通过实际操作，您将直观地看到高并发请求可能导致调用失败或响应延迟，而低并发请求则更为稳定。

> **警告**
> 
> 运行实验代码可能会产生费用，请您在知悉费用详情后再进行实验。
> 
> 截至2024年9月30日，示例中使用的通义千问2.5开源模型（Qwen2.5-0.5B-Instruct）处于限时免费状态，请您前往以下链接确认最新的模型价格。
> 
> [阿里云帮助文档 - 模型入门](https://help.aliyun.com/zh/model-studio/getting-started/models)


#### ⚙️ 设置环境变量

安装依赖

In [3]:
! pip install -r requirements.txt

In [ ]:
# 环境变量
os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("请输入你的api_key:")

# 导入key
import dashscope
dashscope.api_key = os.environ["DASHSCOPE_API_KEY"]

##### 实验目标
- 理解峰谷时段：了解高并发（峰时）和低并发（谷时）对应用的影响。
- 测量成功率和响应时间：记录不同并发量下应用响应的成功率和平均响应时间。
- 识别应用的并发限制：通过实验确定应用在不同并发量下的表现，了解其最大承载能力。

##### 实验代码

以下代码定义了一个小实验，通过模拟不同并发量下通义千问模型调用API的响应，测量在峰谷时段下的调用成功率和平均响应时间。


In [6]:
import os
import asyncio
from openai import AsyncOpenAI
import platform
import time

# 创建 AsyncOpenAI 客户端
client = AsyncOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),  # 替换为您的API Key
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",  # DashScope服务的base_url
)

# 定义异步请求的函数
async def make_request():
    try:
        start_time = time.time()
        completion = await client.chat.completions.create(
            model="qwen2.5-0.5b-instruct",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': '你是谁？'}
            ]
        )
        response_time = time.time() - start_time
        return (True, response_time)
    except Exception as e:
        return (False, str(e))

# 并发测试函数
async def test_concurrency(max_concurrent, duration, results):
    async def worker(worker_id):
        nonlocal results
        end_time = time.time() + duration
        while time.time() < end_time:
            success, result = await make_request()
            if success:
                results['success'] += 1
                results['total_time'] += result
            else:
                results['fail'] += 1
                print(f"Worker {worker_id} Failed: {result}")
            await asyncio.sleep(1)  # 每个worker每1秒发送一个请求

    # 创建并启动所有worker任务
    tasks = [asyncio.create_task(worker(i)) for i in range(max_concurrent)]
    await asyncio.gather(*tasks)

# 定义负载曲线，模拟峰谷时段的请求，每个步骤持续5秒
load_curve = [
    {'concurrent': 1, 'duration': 5},    
    {'concurrent': 2, 'duration': 5},   
    {'concurrent': 4, 'duration': 5},   
    {'concurrent': 2, 'duration': 5},   
    {'concurrent': 1, 'duration': 5},    
]

# 执行负载测试的主函数
async def main():
    print("开始负载测试...\n")
    for step in load_curve:
        concurrent = step['concurrent']
        duration = step['duration']
        results = {'success': 0, 'fail': 0, 'total_time': 0.0}
        print(f"测试并发量: {concurrent}，持续时间: {duration} 秒")
        start_time = time.time()
        await test_concurrency(concurrent, duration, results)
        actual_duration = time.time() - start_time
        total_requests = results['success'] + results['fail']
        avg_time = results['total_time'] / results['success'] if results['success'] > 0 else 0
        print(f"并发量: {concurrent}, 总请求: {total_requests}, 成功: {results['success']}, 失败: {results['fail']}, "
              f"平均响应时间: {avg_time:.2f}s, 实际持续时间: {actual_duration:.2f}s\n")
    print("负载测试完成。")

# 兼容Windows事件循环
if platform.system() == "Windows":
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

# 运行主函数
if __name__ == "__main__":
    # Get the Tkinter event loop
    loop = asyncio.get_event_loop_policy().get_event_loop() 
    # Schedule the coroutine
    loop.create_task(main())  # Or loop.run_until_complete(my_async_function()) for immediate execution


##### 实验结果

执行这段代码后，你可能会看到如下输出：

开始负载测试...

测试并发量: 1，持续时间: 5 秒
并发量: 1, 总请求: 3, 成功: 3, 失败: 0, 平均响应时间: 0.67s, 实际持续时间: 5.00s

测试并发量: 2，持续时间: 5 秒
并发量: 2, 总请求: 7, 成功: 7, 失败: 0, 平均响应时间: 0.98s, 实际持续时间: 7.53s

测试并发量: 4，持续时间: 5 秒
Worker 1 Failed: Error code: 429 - {'error': {'message': 'You exceeded your current requests list.', 'type': 'limit_requests', 'param': None, 'code': 'limit_requests'}}
并发量: 4, 总请求: 12, 成功: 11, 失败: 1, 平均响应时间: 1.73s, 实际持续时间: 9.22s

测试并发量: 2，持续时间: 5 秒
并发量: 2, 总请求: 5, 成功: 5, 失败: 0, 平均响应时间: 1.32s, 实际持续时间: 6.04s

测试并发量: 1，持续时间: 5 秒
并发量: 1, 总请求: 4, 成功: 4, 失败: 0, 平均响应时间: 0.61s, 实际持续时间: 6.45s

你可以看到，当用户的总请求数开始逐渐增多后，大模型的平均响应时间不断增长。如果短时间的总请求数超过了模型服务的流量限制，用户的请求就会失败，这极大影响了用户的使用体验。

##### 实验总结

这是所有高并发应用都可能遇到的挑战。当并发请求大量涌入时，服务器可能会因为超负荷运行而导致资源枯竭，结果是用户体验的下降甚至系统崩溃。这种问题往往在开发阶段不易察觉，但一旦部署到真实环境中，问题就会被放大。

为了解决这些问题，你可以采用阿里云百炼提供的模型部署服务，在负载高峰期大幅提高大模型服务的承载能力，以应对突发的用户需求；在负载低谷期降低大模型服务的资源占用，以尽量减少部署开支。

有关百炼的模型部署方法，我们将在本节课程的第三小节介绍。现在，让我们先来看看什么是模型部署。


### 1.2. 什么是模型部署

模型部署是将训练好的机器学习模型引入生产环境的过程，让它能在真实世界中为各种应用提供智能支持。部署后的模型能够实时处理新数据，输出预测结果，为业务决策提供可靠依据。

模型部署主要有以下功能：

- 提供推理服务
    - 预测推理：当模型部署完成后，它就像一个沉默的智者，时刻准备接受新问题并给出答案。这一过程被称为推理（Inference），是模型在生产环境中最常见的应用形式。
    - API接口：为了方便与外部系统的交流，我们通常会开发API（应用程序接口），通过这些接口，其他系统可以调用模型服务，获取预测结果。常用的API形式包括RESTful和gRPC。
    
- 提高并发量
    - 可扩展性：面对高并发请求，部署架构必须具备良好的可扩展性，以确保在流量高峰时能够快速响应。这就像在高峰期增加收银台，以应对排队的顾客。
    - 负载均衡：通过负载均衡技术，可以将请求均匀分配到多个实例中，从而避免单点故障，提高系统的稳定性和响应速度。
  
- 加速推理过程
    - 高效计算：使用先进的加速技术（如GPU加速、量化模型）能够显著提升模型推理的速度。就像给汽车安装了涡轮增压器，使其更快、更高效。
    - 缓存策略：合理使用缓存可以减少重复计算，提高响应效率。对于一些常见的请求结果，可以提前计算并缓存，以便快速返回结果。
    - 优化算法：通过模型优化技术（如剪枝、蒸馏）可以提高推理效率，使模型更轻量化、更快速。
  
接下来，你将深入理解这些功能背后的技术。

### 1.3. 深入理解模型部署
#### 量化（Quantization）

- 概念：量化是将模型权重和激活值从高精度（如32位浮点数）转换为低精度（如8位整数）表示的一种技术。量化可以显著减少模型的存储和计算资源需求，同时在推理过程中提高速度和节省内存。
- 优势：

    - 计算加速：低精度计算通常比高精度计算更快，尤其是在硬件（如GPU、TPU）上有专门的优化。
    
    - 资源节省：降低内存使用和存储空间，使模型更适合部署在资源受限的设备上（如移动设备和嵌入式系统）。
    
- 应用场景：适用于需要高效推理的场景，如实时应用、嵌入式系统等。

#### 蒸馏（Distillation）
- 概念：模型蒸馏是一种通过训练一个较小的学生模型来模仿较大教师模型的方法。教师模型通常是一个性能优越但复杂度较高的大模型，而学生模型则较小且简化，但通过蒸馏，它能够接近教师模型的性能。

- 优势：

    - 模型压缩：减少模型的参数和计算复杂度，使其更轻量化。
    
    - 推理加速：学生模型通常比教师模型更快，适合在推理阶段使用。
    
- 应用场景：适用于需要部署在计算资源有限的环境中的模型，如移动端应用、边缘计算设备等。

#### 缓存（Caching）

- 概念：缓存是一种通过存储计算结果来减少重复计算、提高响应效率的技术。对于一些常见的请求结果，可以提前计算并存储在缓存中，以便快速返回结果。

- 优势：

    - 减少重复计算：提高系统的响应速度和效率。

    - 降低资源消耗：减少计算资源的使用，优化系统性能。

- 应用场景：适用于高频请求和需要快速响应的应用，如Web服务、推荐系统等。


#### 负载均衡（Load Balancing）

- 概念：负载均衡是一种通过将网络请求均匀分配到多个服务器实例上，从而提高系统的稳定性和响应速度的技术。

- 优势：

    - 避免单点故障：提高系统的可靠性和可用性。

    - 优化资源使用：均衡负载，避免某个实例过载。

- 实现方式：常用的负载均衡方案包括硬件负载均衡器、软件负载均衡器（如Nginx、HAProxy）和云服务提供的负载均衡（如AWS ELB、阿里云SLB）。

#### 数据并行和模型并行

##### 数据并行（Data Parallelism）：

+ 概念：将数据集划分成多个小批次，并行地在多个计算节点上处理相同的模型。这种方法适用于训练和推理的过程，能够显著加快计算速度。

+ 优势：

    + 加速计算：通过并行处理多个数据批次，提高整体计算速度。

    + 负载均衡：均匀分配计算任务，避免单个节点的过载。

+ 应用场景：适用于大规模数据处理和模型训练，如深度学习训练过程。

##### 模型并行（Model Parallelism）：

+ 概念：将模型的不同部分（如不同层或模块）分布在不同的计算节点上进行处理。这种方法适用于超大模型的部署，能够有效利用多个计算资源。

+ 优势：

    + 处理大模型：适用于超大模型，使其能够在多个节点上协同运行。

    + 资源优化：通过分布式计算，充分利用各节点的计算能力。

+ 应用场景：适用于超大规模模型的推理和训练，如GPT-3等超大语言模型。

#### 水平扩展和垂直扩展

##### 水平扩展（Scale-Out）：

+ 概念：通过增加更多的服务器实例来分担负载。常见的方法是使用云服务的弹性伸缩功能，根据流量自动增加或减少实例数量。

+ 优势：
    + 灵活性高：可以动态调节实例数量，适应流量变化。

    + 高可用性：通过多实例部署，提高系统的容错能力。

+ 应用场景：适用于需要处理高并发请求的应用，如Web服务、在线游戏等。

##### 垂直扩展（Scale-Up）：

+ 概念：通过提升单个服务器的硬件性能，如增加CPU核数、内存容量等，来提高处理能力。

+ 优势：

    + 实现简单：不需要修改应用程序架构。

    + 性能提升：直接提升单个节点的处理能力。

+ 应用场景：适用于需要提升单个节点性能的应用，如数据库服务器、计算密集型任务等。

#### RESTful API与OpenAI API兼容

##### RESTful API：

+ 概念：基于HTTP协议，使用标准的HTTP方法（GET、POST、PUT、DELETE）进行通信。RESTful API以资源为中心，通过URL定位资源，并使用HTTP方法对资源进行操作。

+ 优势：

    + 简单易用：符合Web标准，易于集成和使用。

    + 广泛支持：被大多数编程语言和框架支持。

+ 应用场景：适用于各种Web服务和应用程序，例如电商、社交媒体等。

##### OpenAI API兼容：

+ 概念：部分模型服务提供与OpenAI API兼容的接口，方便用户迁移和整合现有的应用程序。这种接口通常基于HTTP或HTTP/2协议，使用JSON格式传输数据。

+ 优势：

    + 易于集成：与现有的OpenAI API兼容，减少迁移成本。

    + 高效通信：支持HTTP/2协议，具有低延迟和高吞吐量的特点。

+ 应用场景：适用于开发使用大语言模型的应用，如智能问答、文本生成等。

通过深入了解这些重要概念，您将能够在实际项目中更有效地优化模型的性能，提高系统的响应速度和处理能力。

接下来，你将掌握部署本地模型（基础模型或微调模型）的方法，以及了解如何在阿里云上部署这些模型，从而享受云服务的便利。


## 🚀 2. 部署本地模型

在开始部署本地模型之前，首先需要规划好硬件资源的使用计划。不同的模型对计算资源有不同的需求，特别是在GPU显存和计算能力方面。

### 2.1. 硬件资源与模型参数量

在部署开源模型时，必须考虑模型的参数量和计算资源需求。参数量越大，对GPU的显存和计算能力要求越高。以下是常见的开源模型如Qwen、Llama等在不同参数量级别下的GPU资源需求：

| 模型类型       | 参数规模            | 部署建议                                    | 适用GPU示例                       |
| -------------- | ------------------- | ------------------------------------------- | -------------------------------- |
| 小型模型       | 7B参数以下          | 可以使用单个GPU进行部署。                   | NVIDIA T4、V100                  |
| 中型模型       | 7B-13B参数          | 建议使用显存较大的GPU以满足模型的需求。     | NVIDIA V100 32GB、A100 40GB      |
| 大型模型       | 30B参数以上         | 需要多GPU进行分布式部署，或者使用显存非常高的GPU。 | NVIDIA A100 80GB                 |

如何快速部署一款开源模型或微调模型呢？vLLM可能是一个不错的选择。

### 2.2. vLLM 是什么

vLLM (Virtual Language Model) 是一种高性能的语言模型推理引擎，专为优化大语言模型（LLM）的运行效率而设计。vLLM 的核心目标是通过改进缓存策略、优化内存管理、提升计算资源利用率，显著加速语言模型的推理速度。它适用于在生产环境中部署大语言模型，尤其是需要高并发处理能力和低延迟的应用场景。

#### vLLM 的特点

- **高效的并发管理：** vLLM 能够同时处理大量的推理请求，支持高并发的推理任务，极大地提高了吞吐量，特别适用于对话式AI、实时问答系统等场景。

- **内存优化与碎片管理：** vLLM 通过先进的内存分配和释放机制，减少内存碎片，提高内存利用率，显著降低运行大模型时的内存占用。

- **改进的缓存策略：** vLLM 引入了创新的缓存技术，能够高效地重用推理过程中生成的中间计算结果，从而减少不必要的重复计算，提升模型运行速度。

- **灵活的模型部署：** vLLM 支持多种语言模型的部署，包括基础模型和经过微调的模型，能够适应不同的应用需求。

#### 如何使用vLLM部署一个开源模型

部署一个基础模型使用 vLLM 包含以下步骤：

1. **安装 vLLM：**

    安装 vLLM 需要确保环境中有  和 （如果使用 GPU）。安装命令如下：

    `pip install vllm`

2. **加载开源模型并启动服务：**

    选择一个开源模型，如 Qwen、Llama、BERT 等。以下示例以 Qwen2.5-7B-Instruct 模型为例：

    `vllm serve "Qwen/Qwen2.5-7B-Instruct"`

#### 如何使用vLLM部署一个微调模型

微调后的模型常以`safetensors`格式保存，vLLM 同样支持这种模型的加载和部署。
1. **安装 vLLM 和 safetensors 支持库：**

    `pip install vllm safetensors`


2. **加载和部署微调后的模型：**

    `vllm serve --model path/to/your-model.safetensors --port 8000`

3. **优化推理：**

    你可以结合多 GPU 或分布式推理的方式，进一步提升推理速度和处理能力。

下面，你将使用 vLLM 部署在之前课程的微调模型。

### 2.3. 在 ModelScope Notebook 上使用 vLLM 部署本地模型服务

在本实验中，你将学习如何使用 ModelScope Notebook 提供的免费 GPU 实例环境，使用 vLLM 部署一个本地的模型服务。

通过实际操作，你将体验到 vLLM 是如何部署本地模型，提供 OpenAI API 兼容的推理服务。

#### 获取 ModelScope Notebook 限时免费 GPU 环境

ModelScope Notebook 为每位新注册用户提供了限时免费的 GPU 测试环境，方便开发者快速上手大模型应用开发。

本次实验我们将使用 ModelScope Notebook 环境作为实验环境。

[ModelScope - 我的笔记本](https://www.modelscope.cn/my/mynotebook/preset)

#### 安装与环境准备

+ 安装 vLLM：在终端中安装 vLLM 及其相关依赖：
+ 设置环境变量：如果使用 ModelScope 模型，可以设置环境变量以确保 vLLM 使用 ModelScope：

#### 部署模型为 OpenAI 兼容服务
+ 启动服务：vLLM 提供了一个 OpenAI 兼容的服务器模式，能够直接部署微调模型。例如，使用以下命令来启动服务：

    其中，path/to/your-model/为模型数据和配置文件的存放目录。



In [ ]:
import subprocess

# 启动 vLLM 服务
def start_vllm_service(model_path, load_format, port):
    command = ["vllm", "serve", model_path, "--load-format", load_format, "--port", str(port)]
    process = subprocess.Popen(command)
    return process

model_path = "/path/to/your/model/"  # 确保这是一个有效的模型目录路径
load_format = "safetensors"  # 确保使用正确的加载格式
port = 8000

print("启动 vLLM 服务...")
vllm_process = start_vllm_service(model_path, load_format, port)

+ 配置 Docker 部署（可选）：你也可以使用 vLLM 提供的官方 Docker 镜像进行部署，确保你拥有合适的 GPU 驱动和共享内存配置：此 Docker 镜像可以将模型部署为一个 OpenAI API 兼容的服务，同时利用 vLLM 的高效推理能力

In [ ]:
docker run --runtime nvidia --gpus all \
    -v ~/.cache/huggingface:/root/.cache/huggingface \
    -p 8000:8000 \
    vllm/vllm-openai:latest \
    --model path/to/your-model/

#### 测试部署效果

+ 发送测试请求：通过 curl 或 Postman 等工具，发送测试请求来验证部署是否成功。例如，可以使用以下 curl 命令测试模型推理效果：通过该命令，你可以验证模型是否正确处理并返回 OpenAI API 兼容的响应格式

In [ ]:
import requests
import json
import time

# 发送测试请求
def send_test_request(port):
    url = f"http://localhost:{port}/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "/path/to/your/model/",
        "messages": [
            {"role": "system", "content": "你是一个帮助用户的助手。"},
            {"role": "user", "content": "请告诉我2020年中国男篮联赛（CBA）的总冠军是谁？"}
        ]
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    return response.json()

port = 8000

# 等待服务启动
print("等待服务启动...")  # 根据实际情况调整等待时间

# 发送测试请求
print("发送测试请求...")
try:
    response = send_test_request(port)
    print("响应结果:")
    print(json.dumps(response, indent=4, ensure_ascii=False))
except Exception as e:
    print(f"请求失败: {e}")

实验成功后，可以将相同的部署方法应用到阿里云 ECS 实例、PAI-EAS 实例或其他服务器环境中，为你的本地模型提供高效的推理服务。根据不同的环境需求，调整配置和资源分配策略以优化模型的性能和稳定性。

### 2.4. 阿里云平台选择（ECS、PAI、FC）

根据应用场景、成本与复杂性对ECS、PAI和FC进行对比，并结合vLLM作为部署工具。

以下是 ECS、PAI 和 FC 的对比表格：

| 对比维度                | ECS (Elastic Compute Service)                          | PAI (Platform for AI)                           | FC (Function Compute)                             |
|------------------------|------------------------------------------------------|------------------------------------------------|-------------------------------------------------|
| 时间（部署复杂性）      | 复杂，需深入理解配置，适合有经验的用户               | 较简单，通过图形化界面操作，自动化程度高       | 最简单，无需管理底层服务器，部署速度快          |
| 成本                   | 较低，长期使用可通过预留实例优化成本                 | 自动扩展功能优化成本，但总体成本略高           | 按需计费，轻量任务成本低                        |
| 稳定性                 | 高，适合长期、稳定运行的服务                          | 高，支持大规模并发和高效管理                   | 高，适合短时、高并发任务                         |
| 性能与扩展性            | 提供全面控制，易于水平和垂直扩展                      | 自动化扩展，适合大规模推理任务                 | 自动伸缩，但对高性能任务支持有限                |
| 开发和部署的复杂性      | 复杂但灵活，适合有经验的团队                         | 较简单，适合快速上线                           | 最简便，适合快速开发和测试                       |
| 安全与合规性            | 高，支持 RAM、VPC 隔离等高级安全功能                   | 中等，支持版本控制和访问管理                   | 基础安全配置，适合低要求任务                     |
| 运维管理与自动化        | 需手动运维，运维工作量较大                            | 自动化管理，减少运维负担                       | 最轻，基础设施管理由平台负责                     |
| 集成与生态兼容性        | 兼容性好，与其他云服务无缝集成                        | 与 AI 工具链深度集成，支持 CI/CD               | 适合无服务器架构，易与其他无服务器服务集成      |

这个表格总结了每个平台的主要优缺点，帮助你根据具体的业务需求和部署场景选择合适的部署平台。根据不同的对比维度，可以更精准地定位最佳选择以实现模型的高效部署。

### 2.5. 使用vLLM部署模型到阿里云 ECS
#### 总体架构

在阿里云 ECS 上部署 vLLM 时，系统架构主要由以下部分组成：

+ ECS 实例：作为主要的计算节点，提供 GPU 计算能力支持大语言模型的推理。

+ vLLM 推理引擎：安装在 ECS 实例上，负责加载和执行语言模型的推理任务。

+ 容器管理与服务层：可以使用 Docker 来管理模型部署，确保推理服务稳定运行。

+ 网络与安全组：配置网络安全组以开放必要端口，确保推理服务的外部可访问性，同时保证系统的安全性。

+ 监控与日志系统：通过阿里云监控服务（如 CloudMonitor）来实时监控 GPU 实例的性能与运行状态，及时发现并处理异常。

#### 注意事项

+ 机型选择：根据模型的参数量和计算需求，推荐选择适配 GPU 的实例类型。常用的 GPU 如 NVIDIA Tesla T4，适合中小型模型部署；对于大型模型或高性能需求，推荐使用 NVIDIA A100。

+ 成本考量：GPU 实例的成本较高，需要根据实际的并发需求和业务负载合理选择实例规格。建议在非高峰期缩减 GPU 实例数量，或使用按需计费模式，避免长期高昂的固定成本。

+ 端口配置：在阿里云控制台中，配置 ECS 安全组规则，开放模型服务使用的端口（如 5000），确保外部用户可以正常访问。

+ 白名单 IP：通过安全组设置白名单 IP，限制仅授权用户访问服务，增强安全性。

+ 授权机制：可以结合 API Key、OAuth 等方式控制用户访问权限，避免未经授权的调用。

+ 监控与告警：使用阿里云 CloudMonitor，监控 GPU 使用率、内存占用和响应时间，并配置告警策略，保障服务的稳定运行。

#### 操作步骤

具体的部署方法可以参考：

[阿里云帮助文档 - 使用 vLLM 容器镜像在 GPU 加速实例上运行推理任务](https://help.aliyun.com/zh/egs/use-cases/use-a-vllm-container-image-to-run-inference-tasks-on-a-gpu-accelerated-instance)

### 2.6. 部署到阿里云 PAI - EAS

#### 总体架构

在阿里云 PAI-EAS 平台上使用 vLLM 部署本地模型时，系统架构包括以下主要部分：

+ EAS 服务实例：使用 EAS（Elastic Algorithm Service）作为模型推理平台，管理和提供高并发、低延迟的模型服务。

+ vLLM 推理引擎：vLLM 提供优化的推理引擎，用于高效地运行大语言模型，支持并发管理和内存优化。

+ 模型与数据存储：通过阿里云的 OSS（对象存储）或 NAS，将模型权重和代码挂载到 EAS 实例，确保数据访问和更新的便利性。

+ Web 接口与 API 服务：部署完成后，EAS 生成的 Web 应用或 API 服务供用户交互，支持实时请求和响应处理。

+ 监控与日志系统：使用 PAI 的监控功能，对模型服务的性能进行实时监控和优化。

#### 注意事项

+ 资源配置：EAS 支持多种 GPU 实例，常见选择包括 ，适用于对性能要求较高的推理任务。可以根据业务需求选择合适的实例类型，同时启用推理加速功能（如 BladeLLM、vLLM）以提升并发性能。

+ 成本控制：建议在负载较高时使用专用资源组，并在低负载时自动缩减资源以控制成本。使用抢占式实例也可以降低长时间运行的费用。

+ 端口与网络安全：EAS 自动配置推理服务的端口，可通过阿里云控制台调整网络安全组规则，限制对服务的访问权限。

+ 访问权限控制：可使用 API Key、OAuth 认证等方式控制用户访问，并设置白名单 IP 以防止未经授权的访问。

+ 监控与告警：启用 PAI-EAS 的监控功能，实时查看服务性能和资源使用情况，并设置告警策略以应对突发情况

#### 操作步骤

要在PAI-EAS上使用vLLM部署，您可以遵循以下步骤：

+ 选择部署方式：

    + 登录PAI控制台，进入相应的工作空间。

    + 导航至“模型部署 > 模型在线服务（EAS）”页面，点击“部署服务”。

+ 自定义模型部署：

    + 在“自定义模型部署”区域，点击“自定义部署”，进入“新建服务”页面。

+ 配置服务信息：

    + 服务名称：输入自定义的模型服务名称。

    + 部署方式：选择“镜像部署AI-Web应用”。

+ 选择镜像：

    + 镜像选择：在PAI平台镜像列表中，选择“chat-llm-webui”，确保镜像版本选择为支持vLLM的版本，如“3.0-vllm”。

+ 模型配置（如需挂载自定义模型）：

    + 点击“填写模型配置”，配置OSS挂载路径，例如模型文件所在OSS路径为。

    + 挂载路径设置为，确保与模型配置中的挂载路径一致。

    + 是否只读开关通常关闭，除非有特殊需求。

+ 推理加速配置：

    + 在资源配置区域，找到“推理加速”参数，选择“开源框架vllm推理加速”。

+ 完成配置并部署：

    + 审核所有配置信息无误后，点击“部署”按钮启动部署流程。

通过以上步骤，您就可以在PAI-EAS平台上利用vLLM推理加速引擎部署大语言模型了。请注意，使用vLLM推理加速时，EAS服务将不支持LangChain功能。此外，确保模型配置与所选推理加速引擎兼容，以达到最佳性能效果。

**参考链接：**

[LLM大语言模型 部署EAS服务]( https://help.aliyun.com/zh/pai/user-guide/deploy-an-llm/)

[5分钟使用EAS一键部署LLM大语言模型应用 常见问题及解决方法]( https://help.aliyun.com/zh/pai/use-cases/deploy-llm-in-eas)

[LLM智能路由：提升LLM推理系统整体效率 部署服务]( https://help.aliyun.com/zh/pai/user-guide/use-llm-intelligent-router-to-improve-inference-efficiency)


### 2.7. 部署到阿里云 FC

本节介绍如何在阿里云函数计算（FC）平台上部署本地模型

#### 总体架构

在阿里云函数计算（FC）平台上使用本地模型时，系统架构主要由以下几个部分组成：

+ 函数计算实例：作为主要的执行环境，负责处理所有的模型推理请求。FC 支持事件驱动的架构，可以灵活地根据请求量自动扩展或收缩实例数量。

+ 模型存储与加载：使用阿里云 OSS（对象存储服务）或直接将模型文件打包到 Docker 镜像中，并在函数启动时加载。这种方法可以有效减少加载延迟并简化模型更新流程。

+ 容器与运行时管理：可以通过自定义 Docker 镜像来定义模型推理所需的依赖环境，并将该镜像部署到 FC 实例中进行高效的模型推理。

+ 触发器与 API 服务：通过 HTTP、定时器、事件等触发器来启动模型推理服务，支持灵活的调用方式和实时的响应处理。

#### 注意事项

+ 实例选择：FC 支持多种实例类型，推荐使用 GPU 加速实例来提升大模型的推理性能。实例类型和配置可在创建服务时指定，根据实际业务需求选择合适的 CPU、内存和 GPU 资源。

+ 成本控制：函数计算采用按需计费模式，可以在高峰期自动扩展实例数量来满足请求，同时在低峰期缩减实例，避免资源浪费。针对 GPU 实例的长时间任务，可以通过预置实例来优化启动时间和降低成本。

+ 端口与安全配置：在 FC 控制台中配置服务的触发器和安全组，确保只开放必要的端口并设置 IP 白名单来防止未经授权的访问。

+ 访问控制：使用 API Key、OAuth 认证或其他安全验证方式控制访问权限，并通过阿里云的监控服务实时跟踪服务性能和安全事件。

+ 监控与告警：启用阿里云 Log Service 和监控功能，对 GPU 实例的资源使用、响应时间等关键指标进行监控，及时响应异常情况。

#### 操作步骤：

要在函数计算的GPU实例上部署模型，您可以遵循以下步骤：

1. 创建函数应用：
    
    登录阿里云，创建一个新的函数应用。选择合适的运行时环境（如Python），并配置相应的函数计算资源，包括实例规格、内存、存储等。

2. 上传或选择镜像：

    如果您已经拥有vLLM的Docker镜像，可以直接在函数计算中上传该镜像；或者，如果镜像已托管在阿里云容器镜像服务或其他支持的镜像仓库中，直接引用该镜像地址。

3. 配置函数：
    
    在函数配置中，指定入口点（如主函数），并设置环境变量，包括模型路径、端口等，确保函数能够正确加载模型和启动服务。

4. 部署并检查：

    部署函数后，检查函数的 镜像加速准备状态 ，确保其变为 可用 。这是函数计算针对大型AI镜像提供的优化功能，有助于缩短启动时间。

5. 测试与验证：

    通过访问函数计算提供的访问域名或API Gateway，发送测试请求以验证vLLM服务是否正常运行，能否正确响应推理请求。

**参考链接** 

+ [实时推理场景 实时推理场景基本信息](https://help.aliyun.com/zh/functioncompute/fc-2-0/use-cases/real-time-inference-scenarios)

+ [准实时推理场景 容器支持]( https://help.aliyun.com/zh/functioncompute/fc-3-0/use-cases/quasi-real-time-inference-scenarios)

+ [实时推理场景 部署方式]( https://help.aliyun.com/zh/functioncompute/fc-3-0/use-cases/real-time-inference-scenarios-1)

+ [准实时推理场景 容器支持]( https://help.aliyun.com/zh/functioncompute/fc-2-0/use-cases/quasi-real-time-inference-scenarios
)
+ [基于函数计算部署3D卡通风格模型 操作步骤]( https://help.aliyun.com/zh/functioncompute/fc-2-0/use-cases/deploy-3d-cartoon-style-model-based-on-function-calculation)
  

## ☁️ 3. 部署线上模型到百炼

在你的机器人项目中也使用了通义千问线上模型。阿里云百炼为这类模型开发了简明的部署服务，你无需关注模型部署的具体实践，也能快速创建部署任务。

### 3.1. 百炼模型部署服务

百炼模型部署服务旨在解决大模型在应对高并发请求时的性能问题。通过独占实例的灵活管理，用户可以动态扩展或缩减模型的并发处理能力，从而保证在流量高峰期也能高效响应用户请求，确保系统稳定运行。同时，用户能够通过API灵活控制成本，降低资源消耗和部署费用。

### 3.2. 部署操作步骤

1. 创建部署服务：通过API创建一个新的模型部署任务，指定模型名称和独占实例数量。
2. 查询部署状态：检查部署任务的状态，确认服务是否已成功运行。
3. 更新部署服务：根据需求动态调整独占实例的数量，增加或减少并发处理能力。
4. 删除部署服务：当不再需要某个部署服务时，通过API删除任务以释放资源。

### 3.3. 模型部署 API
百炼模型部署服务提供了一系列API接口，用户可以通过以下方法来管理模型的部署：

1. 创建部署任务，参数包括模型名称和所需的独占实例数量。

    ```bash
    curl 'https://dashscope.aliyuncs.com/api/v1/deployments' \
        --header 'Authorization: Bearer <YOUR-DASHSCOPE-API-KEY>' \
        --header 'Content-Type: application/json' \
        --data '{        
                    "model_name": "qwen-plus",      
                    "capacity": 8,        
                    "suffix": "sample"
                }'

2. 查询指定部署任务的详细信息，如状态和资源配置。

    ```bash
    curl 'https://dashscope.aliyuncs.com/api/v1/deployments/qwen-plus-sample' \
        --header 'Authorization: Bearer <YOUR-DASHSCOPE-API-KEY>' \
        --header 'Content-Type: application/json'


3. 更新部署任务的实例数量，调整并发处理能力。

    ```bash
    curl --request PUT 'https://dashscope.aliyuncs.com/api/v1/deployments/qwen-plus-sample/scale' \
        --header 'Authorization: Bearer <YOUR-DASHSCOPE-API-KEY>' \
        --header 'Content-Type: application/json' \
        --data '{    
                    "capacity":4
                }'

4. 删除指定部署任务，释放资源。

    ```bash
    curl --request DELETE 'https://dashscope.aliyuncs.com/api/v1/deployments/qwen-plus-sample' \
        --header 'Authorization: Bearer <YOUR-DASHSCOPE-API-KEY>' \
        --header 'Content-Type: application/json'

> **警告：**
> 
> 直接发送 API 请求可能产生大量计费，请先前往阿里云百炼文档，获悉计费规则后再运行实例代码，防止产生额外计费。
> 
> 以部署 qwen-plus 模型为例，每独占实例每小时计费为160元（价格信息基于2024年9月30日，请以最新价格为准）
> 
> [阿里云帮助文档 - Model Studio 计费](https://help.aliyun.com/zh/model-studio/billing-for-model-studio)


详细使用方法请参考：

[阿里云帮助文档 - 模型部署快速入门](https://help.aliyun.com/zh/model-studio/developer-reference/model-deployment-quick-start)

## 🤖 4. 拓展阅读：让机器人开始“上岗”

在经过部署优化后，新人答疑机器人具备了更强的并发处理能力。接下来，我们将提供两个简单的发布方式，方便你了解发布机器人的基本流程：

### 将机器人发布为网站 AI 助手

在阿里云百炼文档中，我们提供了一个快速启动的示例，帮助千万用户快速发布百炼RAG应用到他们自己的网站上，作为 AI 助手来服务客户。现在，你也可以参照这篇最佳实践文档，将本课程中开发的机器人也发布为网站的AI助手，以一种更便捷的形式为新员工提供服务。

[阿里云帮助文档 - 在10分钟内为网站添加 AI 助手](https://help.aliyun.com/zh/model-studio/use-cases/add-an-ai-assistant-to-your-website-in-10-minutes)

### 创建钉钉群AI机器人

我们也提供了“百炼应用发布为钉钉群AI机器人的方法”示例，你也可以参照这篇文档来发布属于自己的钉钉群机器人，让新员工即刻获取回答。

[阿里云帮助文档 - 在10分钟内为钉钉添加 AI 助手](https://help.aliyun.com/zh/model-studio/use-cases/add-an-ai-assistant-to-your-dingtalk-in-10-minutes)

无论是选择发布到网站AI助手还是钉钉机器人，甚至是其他形式的AI应用，你将成功让新人答疑机器人的“上岗”。这将极大提高新员工的使用便利性，并有效降低答疑成本。

## ✅本节小结

本节课程详细介绍了如何通过阿里云百炼的模型部署服务来提升新人答疑机器人的并发处理能力，以及如何将机器人应用发布到实际业务中。你学习了：

+ 了解如何将本地模型部署到阿里云，包括ECS、PAI和FC三种方式。

+ 使用模型部署API创建、查询、更新和删除部署服务。

+ 通过实际操作，提升了机器人在高并发场景下的稳定性。

+ 将机器人发布到生产环境中，方便新员工使用。

通过本节的学习，你已经掌握了从模型部署到应用发布的完整流程，为构建高性能、可扩展的大模型应用打下了坚实的基础。